In [35]:
# Template Notebook for Indvidualized Participant Engagement Classifcation NN Model

# ToDo/Goals: https://docs.google.com/document/d/1_dTnbdRoBV0A7xCxVEyLBdckrlsUnMnRFbbRn39tiXk/edit?usp=sharing
# Data: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [36]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils import np_utils

#### TODO: Specifiy Data Path Based on Participant/Your Computer

In [37]:
# TODO: Specify Data Path based on participant/your computer
# USE data from FS4 folder

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p8_data_FS4.csv'
data = pd.read_csv(file)

In [38]:
data.head()

,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,...,ts_attempt,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
1,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
2,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
3,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
4,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1


In [39]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

1.0    139463
0.0    120473
Name: engagement, dtype: int64

1 0.5365282223316509
0 0.4634717776683491


### Separate Data into Game vs No Game

### Train Test Split (for chosen Feature Set)

#### TODO: Specify which train-test split you want

1. Random Shuffle Split: 70-30, 50-50, 30-70, 10-90
2. Chronological Split: 70-30, 50-50, 30-70
3. Session Split: 70-30, 50-50, 30-70, 10-90

In [40]:
# Session Split: Train on Percentage of Data From Each Session, Sorted Chronologically 

def session_split(data, size):
    sessions = data['session_num'].unique()
    
    first = True
        
    for s in sessions:
        curr = data.loc[data['session_num'] == s]
        curr = curr.sort_values(['timestamp'], ascending=[True])

        y = curr['engagement']
        X = curr.drop(columns=['timestamp', 'session_num', 'engagement'])

        if first:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, shuffle=True)
            first = False
            
        else:
            CX_train, CX_test, Cy_train, Cy_test = train_test_split(X, y, test_size=size, shuffle=True)
                        
            X_train = X_train.append(CX_train)
            X_test = X_test.append(CX_test)
            y_train = y_train.append(Cy_train)
            y_test = y_test.append(Cy_test)
    
    return X_train, X_test, y_train, y_test

In [41]:
# Sort data (don't need for Random Shuffle, but ok to leave it)

data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [42]:
# TODO: Uncomment train-test split to run below 

y = data['engagement']
X = data.drop(columns=['timestamp', 'session_num', 'engagement'])

# Choose one of these 


# 1. Random Shuffle Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

### (Extra)
# Random Shuffle Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Shuffle Split 30-70
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Random Shuffle Split 10-90
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)
### 

#############################

# 2. Chronological Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# 3. Chronological Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

# 4. Chronological Split 30-70
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=False)

############################# 

# 5. Session Split 70-30
# X_train, X_test, y_train, y_test = session_split(data, 0.3)

# 6. Session Split 50-50
# X_train, X_test, y_train, y_test = session_split(data, 0.5)

# 7. Session Split 30-70
# X_train, X_test, y_train, y_test = session_split(data, 0.7)

# 8. Session Split 10-90
# X_train, X_test, y_train, y_test = session_split(data, 0.9)

In [43]:
# Sort X,y Train dataframes together after split 

X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

In [44]:
X_train0 = X_train.loc[X_train['no_game'] == 0]
y_train0 = y_train.loc[X_train['no_game'] == 0]

X_train1 = X_train.loc[X_train['no_game'] == 1]
y_train1 = y_train.loc[X_train['no_game'] == 1]

X_test0 = X_test.loc[X_test['no_game'] == 0]
y_test0 = y_test.loc[X_test['no_game'] == 0]

X_test1 = X_test.loc[X_test['no_game'] == 1]
y_test1 = y_test.loc[X_test['no_game'] == 1]

### Step 3: Neural Network Model

In [68]:
# Model for No Game 

num_epochs0 = 1
size_batch0 = 1024   

model0 = Sequential()

model0.add(Dense(units=64, input_dim=(X_train.shape[1]), kernel_initializer='glorot_normal'))
model0.add(Activation('relu'))

model0.add(Dense(units=32, kernel_initializer='glorot_normal'))
model0.add(Activation('relu'))

model0.add(Dense(units=16, kernel_initializer='glorot_normal'))
model0.add(Activation('relu'))

model0.add(Dense(units=1))
model0.add(Activation('sigmoid'))

model0.summary()

model0.compile(loss='binary_crossentropy',  
              optimizer='Adam',
              metrics=['accuracy'])

model0.fit(X_train0, y_train0, epochs=num_epochs0, batch_size=size_batch0, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 64)                1920      
_________________________________________________________________
activation_30 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 32)                2080      
_________________________________________________________________
activation_31 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 16)                528       
_________________________________________________________________
activation_32 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 17        
__________

In [78]:
# Model for Game 

num_epochs1 = 1
size_batch1 = 1   

model1 = Sequential()

model1.add(Dense(units=64, input_dim=(X_train.shape[1]), kernel_initializer='glorot_normal'))
model1.add(Activation('relu'))

model1.add(Dense(units=32, kernel_initializer='glorot_normal'))
model1.add(Activation('relu'))

model1.add(Dense(units=16, kernel_initializer='glorot_normal'))
model1.add(Activation('relu'))

model1.add(Dense(units=1))
model1.add(Activation('sigmoid'))

model1.summary()

model1.compile(loss='binary_crossentropy',  
              optimizer='Adam',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 64)                1920      
_________________________________________________________________
activation_54 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 32)                2080      
_________________________________________________________________
activation_55 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 16)                528       
_________________________________________________________________
activation_56 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 17        
__________

In [69]:
loss_and_metrics = model0.evaluate(X_test0, y_test0)
print(loss_and_metrics)

146238/146238 [==============================] - 4s 28us/step
[0.6622346195718418, 0.6126588164498967]


In [79]:
model1.fit(X_train1, y_train1, epochs=num_epochs1, batch_size=size_batch1, verbose=1)

Epoch 1/1
6866/6866 [==============================] - 15s 2ms/step - loss: 0.3780 - acc: 0.8147


In [80]:
loss_and_metrics = model1.evaluate(X_test1, y_test1)
print(loss_and_metrics)

35718/35718 [==============================] - 1s 41us/step
[2.4395732833112063, 0.2512458704294753]


In [81]:
len(X_test1)

35718